# Using LSTM and a proxy task of paper title predicition to get better Word vectors #

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import json, gensim, sklearn, pickle, sys, re, os
import IPython.display as ipd
from gensim.parsing.preprocessing import preprocess_documents
from gensim.parsing.preprocessing import preprocess_string
from gensim.models.phrases import Phrases
from gensim.parsing.preprocessing import strip_tags, strip_short, strip_multiple_whitespaces, stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim import utils
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from IPython.display import Audio
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
sound_file = './Music/invalid_keypress.mp3'

In [4]:
datapd=pd.read_json("/home/ubuntu/project/arxiv-metadata-oai-snapshot.json", lines=True)

In [5]:
datapd_hep_th = datapd.loc[datapd['categories'].str.contains('hep-th')] 

In [6]:
hep_abstracts = datapd_hep_th[ list(datapd_hep_th.iloc[:,0:1]) + ['abstract'] + ['title']+['authors']]
hep_abstracts_limit=hep_abstracts[hep_abstracts.title.str.count(' ') < 15]

In [7]:
hep_corpus_limit=hep_abstracts_limit['abstract'].values 
hep_corpus_limit_eof=hep_corpus_limit+" EOF"
short_hep_abs=hep_corpus_limit_eof

In [8]:
hep_titles=hep_abstracts_limit['title'].values
hep_titles_eof=hep_titles + " EOF"
short_hep_tit=hep_titles_eof

In [9]:
#processed_hep=preprocess_documents(hep_corpus_limit)
#pickle.dump(processed_hep,open("processed_hep","wb"))
processed_hep=pickle.load(open("/home/ubuntu/project/LSTM/Mar4bigtrainingWord/processed_hep","rb"))

In [10]:
#hep_dictionary_lim = gensim.corpora.Dictionary(processed_hep)
#hep_bow_corpus_lim = [hep_dictionary_lim.doc2bow(text) for text in processed_hep]
#pickle.dump(hep_dictionary_lim,open("hep_dictionary_lim","wb"))
#pickle.dump(hep_bow_corpus_lim,open("hep_bow_corpus_lim","wb"))

In [11]:
hep_dictionary_lim=pickle.load(open("/home/ubuntu/project/LSTM/Mar4bigtrainingWord/hep_dictionary_lim","rb"))
hep_bow_corpus_lim=pickle.load(open("/home/ubuntu/project/LSTM/Mar4bigtrainingWord/hep_bow_corpus_lim","rb"))

In [12]:
hep_tfidf = gensim.models.TfidfModel(hep_bow_corpus_lim, smartirs='npu')

In [13]:
index = gensim.similarities.MatrixSimilarity(hep_tfidf[hep_bow_corpus_lim])

### By doing the above, we have obtained the 130K papers that belong in high energy theory physics (hep-th) ###

### We restrict to paper with titles shorter than 16 words. We also add an " EOF" to indicate the end of the abstract/title ###

## We will now process the documents using gensim. This removes stop words, punctuations, capitalization and stems and lemmatizes the tezt ##

In [ ]:
doc_tokenized_nostop=preprocess_documents(short_hep_abs)
doctitle_tokenized_nostop=preprocess_documents(short_hep_tit)
#pickle.dump(doc_tokenized_nostop,open("Modelsfeb18/doc_tokenized_nostop", 'wb'))
#pickle.dump(doc_tokenized_nostop,open("Modelsfeb18/doctitle_tokenized_nostop", 'wb'))
#doc_tokenized_nostop=pickle.load(open("/home/ubuntu/Modelsfeb18/doc_tokenized_nostop", 'rb'))
#doctitle_tokenized_nostop=pickle.load(open("/home/ubuntu/Modelsfeb18/doctitle_tokenized_nostop", 'rb'))

In [ ]:
leabs = [len(x) for x in doc_tokenized_nostop]
letit = [len(x) for x in doctitle_tokenized_nostop]

In [ ]:
count=0
for x in range(len(leabs)):
    if leabs[x]<=155:
        count=count+1
print(count)

In [ ]:
count=0
for x in range(len(letit)):
    if letit[x]<=12:
        count=count+1
        #print(x)
print(count)

### The above tells us that almost 95% of the total number of hep-th papers (140,500) have abstracts shorter than 155 words and titles shorter than 12 words. We will use this fact below when constructing our LSTM network ###

In [ ]:
print(max(letit),max(leabs))

In [ ]:
doc_tokenized_nostop[225]

In [ ]:
doctitle_tokenized_nostop[225]

In [ ]:
vec_size=52

## Below we train Word2Vec. This will learn word embeddings ## 

The below line trains Word2Vec on all the abstracts

In [ ]:
#short_hepmodel=Word2Vec(sentences=doc_tokenized_nostop,size=vec_size, window=5, min_count=1, workers=4, iter=1)

In [ ]:
short_hepmodel=pickle.load(open("/home/ubuntu/project/LSTM/Modelsfeb18/hepmodel_mar_1_1538", 'rb'))

In [ ]:
totalvocab=len(short_hepmodel.wv.vocab)
print(totalvocab)

The line below further trains the model on all the paper titles

In [ ]:
#short_hepmodel.build_vocab(doctitle_tokenized_nostop, update=True)
#short_hepmodel.train(doctitle_tokenized_nostop, total_examples=short_hepmodel.corpus_count, epochs=2)

In [ ]:
totalvocab=len(short_hepmodel.wv.vocab)
print(totalvocab)

In [ ]:
#pickle.dump(short_hepmodel,open("Modelsfeb18/hepmodel_mar_1_1538", 'wb'))
#short_hepmodel=pickle.load(open('ModelsFeb23/hepmodel_feb_26_1558','rb'))

# Visualising using t-SNE #

In [ ]:
def tsnescatterplot(model, word):
    
    arr = np.empty((0,vec_size), dtype='f')
    word_labels = [word]

    close_words = model.wv.similar_by_word(word,topn=15)
    

    arr = np.append(arr, np.array([model.wv[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model.wv[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        

    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]

    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 5), textcoords='offset points')
    plt.xlim(x_coords.min()-15, x_coords.max()+15)
    plt.ylim(y_coords.min()-15, y_coords.max()+15)
    plt.show()

In [ ]:
tsnescatterplot(short_hepmodel, 'newton')

In [ ]:
short_hepmodel.wv.most_similar(positive=["newton"],topn=15)

# Now we will train an LSTM network with attention to get better word embeddings #

## The aim of the task is to predict paper titles from the abstract. This is an impossibly hard task, but hopefully we will learn good word emebeddings along the way ##

In [ ]:
#!pip install keras  
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
from keras.layers.embeddings import Embedding
import keras.backend as K

## Setting max length of the abstract and title to be 155 and 12 respectively. As shown above this is a reasonable thing to do ##

In [ ]:
Tx=155
Ty=12

### The below function takes in sentences, and out puts a list of numbers. Each number correspond to a word. Thus the range of the numbers is from 0 to length of the vocab.  ###

In [ ]:
def sentences_to_indices(X, hepmodel,max_len):
    m = X.shape[0] 
    X_indices = np.zeros((m,max_len),dtype='uint16')+hepmodel.wv.vocab.get('eof').index
    for i in range(m):                          
        sentence_words = X[i]
        j = 0
        for w in sentence_words:
            X_indices[i, j] = hepmodel.wv.vocab.get(w).index
            j = j+1
            if j>=max_len:
                break
    return X_indices

In [ ]:
array_abs=np.asarray(doc_tokenized_nostop)
array_tit=np.asarray(doctitle_tokenized_nostop)

In [ ]:
train_X_indices=sentences_to_indices(array_abs,short_hepmodel,Tx)
train_Y_indices=sentences_to_indices(array_tit,short_hepmodel,Ty)

In [ ]:
train_X_indices[1]

## The below function creates a small dense network for the attention layer ##

In [40]:
from keras.activations import softmax
def softmax_axis1(x):
    return softmax(x,axis=1)
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(Tx)
densor2 = Dense(Tx)
densor3 = Dense(10)
densor4 = Dense(1, activation = "relu")
activator = Activation(softmax_axis1, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [41]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a,s_prev])

    e = densor1(concat)
    e = tf.keras.layers.LeakyReLU(alpha=0.3)(e)


    e = densor3(e)
    e = tf.keras.layers.LeakyReLU(alpha=0.3)(e)

    energies = densor4(e)
    alphas = activator(energies)
    context = dotor([alphas,a])
    
    return context

In [42]:
X1 = np.array([['witten', 'unif', 'string', 'theori' ],['dirac', 'discover', 'field' , 'theori']])
X1_indices = sentences_to_indices(X1, short_hepmodel,max_len = Tx)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

X1 = [['witten' 'unif' 'string' 'theori']
 ['dirac' 'discover' 'field' 'theori']]
X1_indices =
 [[  426   969    10     0     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1     1     1     1     1     1     1     1     1     1     1
      1     1   

## Now let's set up the LSTM network ##

In [43]:
n_a = 64 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

post_activation_LSTM_cell = LSTM(n_s, return_state = True,return_sequences = True ) 
output_layer = Dense(totalvocab, activation=softmax_axis1)

## We will now set up the Keras embedding layer. We initialize the Keras embedding matrix using our pretrained Word2Vec vectors. ##


In [44]:
small_vectors=short_hepmodel.wv.vectors
appker=np.zeros((1,vec_size))
kerasvectors=np.vstack((small_vectors,appker))

## Define the embedding layer and set *trainable=True*. We will read the weights from this layer once the network has trained. ##

In [45]:
def pretrained_embedding_layer(hepmodel):
    vocab_len = totalvocab + 1                  
    emb_dim = hepmodel.wv["newton"].shape[0]      
    emb_matrix = np.zeros((vocab_len,emb_dim))
    embedding_layer = Embedding(vocab_len,emb_dim,trainable=True)
    embedding_layer.build((None,)) 
    embedding_layer.set_weights([kerasvectors])
    
    return embedding_layer

In [46]:
embedding_layer = pretrained_embedding_layer(short_hepmodel)
print("weights[0][1628][12] =", embedding_layer.get_weights()[0][1628][12])

weights[0][1628][12] = -0.15372208


In [47]:
train_Y_indices

array([[  83,  491,   63, ...,    1,    1,    1],
       [4041,    4, 8213, ...,    1,    1,    1],
       [   6,    2,    0, ...,    1,    1,    1],
       ...,
       [3399,  955,  541, ...,    1,    1,    1],
       [ 158, 3117,  541, ...,   91,    1,    1],
       [  10,    3, 2707, ...,    1,    1,    1]], dtype=uint16)

## Define the model. 1 bi-directional LSTM --> Attention layer ---> LSTM ----> Softmax. The output is (Ty) number of column vectors of dimension (totalvocab).

In [48]:
tf.keras.backend.clear_session()

In [61]:
def kerasmagic(input_shape, Tx, Ty, n_a, n_s, totalvocab, hepmodel): 
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    contextual = []
    hidden_state = []
    sentence_indices = Input(shape=input_shape, dtype='int32')
    
    # Create the embedding layer 
    embedding_layer = pretrained_embedding_layer(hepmodel)
    embeddings = embedding_layer(sentence_indices)
    
    
    b = Bidirectional(LSTM(units=n_a, return_sequences=True))(embeddings)
    
    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b)
    
    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b)
    
    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b)
    
    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b)

    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b)
    
    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b)
    
    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b)
    
    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b)
    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b)
    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b)
    #b = Bidirectional(LSTM(units=n_a, return_sequences=True))(b) 
    
        
    for t in range(Ty):
    
        # Perform one step of the attention mechanism #
        context = one_step_attention(b,s)
        contextual.append(context)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        _, s , c = post_activation_LSTM_cell(context, initial_state=[s,c])
        hidden_state.append(s)
        out = output_layer(s)
        outputs.append(out)
        
        
    array_out=tf.convert_to_tensor(outputs)
    array_outputs=tf.transpose(array_out,[1,0,2])


    model1 = Model([sentence_indices,s0,c0],array_outputs)
    model2 = Model([sentence_indices,s0,c0],[outputs,hidden_state,contextual])
    
    return model1, model2

In [62]:
kerasmodel=kerasmagic(Tx,Tx,Ty,n_a,n_s,totalvocab,short_hepmodel)

In [63]:
kerasmodel[0].summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 155)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 155, 52)      1864408     input_3[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 [(None, 64)]         0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 155, 128)     59904       embedding_2[0][0]                
____________________________________________________________________________________________

In [64]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.0)
kerasmodel[0].compile(optimizer=opt, loss=['categorical_crossentropy'], metrics=['accuracy'])
kerasmodel[1].compile(optimizer=opt, loss=['categorical_crossentropy','categorical_crossentropy'], metrics=['accuracy'],loss_weights=[1,0])

# Note we compile two Keras models. But only train the first one. The second model is to just used to obtain the contextual and hidden states #

In [65]:
short_hepmodel.wv.index2word[0]

'theori'

In [66]:
print(train_X_indices.size*train_X_indices.itemsize)
print(train_Y_indices.size*train_Y_indices.itemsize)

41631760
3223104


### echo 1 | sudo tee /proc/sys/vm/overcommit_memory ###

In [69]:
vocab_size = totalvocab
text_length = len(train_Y_indices[0])
one_hot = np.zeros([train_X_indices.shape[0],vocab_size,text_length],dtype='uint16')

In [70]:
for i in range(train_X_indices.shape[0]):
    for j in range(Ty):
        one_hot[i,train_Y_indices[i,j],j]=1

In [71]:
listhot = list(one_hot.T.swapaxes(1,2))

In [72]:
print(one_hot.shape)
print(train_X_indices.shape)
print(listhot[7].shape)

(134296, 35853, 12)
(134296, 155)
(134296, 35853)


In [73]:
from sklearn.utils import shuffle
X_shuffled, Y_shuffled = shuffle(train_X_indices,train_Y_indices)

In [74]:
one_hot_shuffled = np.zeros([X_shuffled.shape[0],vocab_size,text_length],dtype='uint16')
for i in range(X_shuffled.shape[0]):
    for j in range(Ty):
        one_hot_shuffled[i,Y_shuffled[i,j],j]=1

# We now use a batch generator, since the system runs out of memory otherwise #

In [75]:
exp_size=16384
def BatchGenerator(X_shuffled,one_hot_shuffled):
    for i in range(1,9):
        X_train = X_shuffled[exp_size*(i-1):exp_size*i,:]
        one_hot_sliced_shuffled = one_hot_shuffled[exp_size*(i-1):exp_size*i,:,:]
        Y_train = list(one_hot_sliced_shuffled.T.swapaxes(1,2))
        yield (X_train, Y_train)
    X_train = X_shuffled[exp_size*i:-1,:]
    one_hot_sliced_shuffled = one_hot_shuffled[exp_size*i:-1,:,:]
    Y_train = list(one_hot_sliced_shuffled.T.swapaxes(1,2))
    yield (X_train,Y_train)

In [57]:
i=0
for (X_train,Y_train) in BatchGenerator(X_shuffled,one_hot_shuffled):
    m = X_train.shape[0]
    s0 = np.zeros((m, n_s))
    c0 = np.zeros((m, n_s))
    Yarraytemp=np.array(Y_train)
    Yarray=np.swapaxes(Yarraytemp,0,1)
    i=i+1
    print(f" \n Metabatch number --------------------- {i} \n ")
    kerasmodel[0].fit([X_train, s0, c0], Yarray, epochs=1, batch_size=128)

 
 Metabatch number --------------------- 1 
 
128/128 [==============================] - 49s 382ms/step - loss: 4.6583 - accuracy: 0.1651
 
 Metabatch number --------------------- 2 
 
128/128 [==============================] - 48s 374ms/step - loss: 4.6250 - accuracy: 0.1673
 
 Metabatch number --------------------- 3 
 
128/128 [==============================] - 48s 373ms/step - loss: 4.5706 - accuracy: 0.1691
 
 Metabatch number --------------------- 4 
 
128/128 [==============================] - 51s 398ms/step - loss: 4.5204 - accuracy: 0.1706
 
 Metabatch number --------------------- 5 
 
128/128 [==============================] - 48s 374ms/step - loss: 4.5245 - accuracy: 0.1715
 
 Metabatch number --------------------- 6 
 
128/128 [==============================] - 48s 374ms/step - loss: 4.4926 - accuracy: 0.1737
 
 Metabatch number --------------------- 7 
 
128/128 [==============================] - 48s 376ms/step - loss: 4.4626 - accuracy: 0.1757
 
 Metabatch number -------

## After having trained the model. We have two choices available to create abstract vectors. The first is to use the finetuned word vectors and the second is to use the hidden states ##

## Unfortunately the Hidden states do not lead to meaninful predictions, and so we now focus on the word vectors. Neverthless the code for using Hidden states can be found at the end of this notebook ##

In [58]:
new_weights = kerasmodel[0].layers[1].get_weights()[0]

# Store the weights #

In [67]:
pickle.dump(new_weights,open("Weights_v3",'wb'))

## Here we define functions to check the ditances between word vectors. Old vs new distance corresponds pre and post fine tuning. While random distance corresponds to a random initialization. ## 

In [76]:
def randomdistance(w1,w2): 
    num=np.dot(kerasvectors[short_hepmodel.wv.vocab.get(w1).index,:],kerasvectors[short_hepmodel.wv.vocab.get(w2).index,:])
    den1=np.dot(kerasvectors[short_hepmodel.wv.vocab.get(w1).index,:],kerasvectors[short_hepmodel.wv.vocab.get(w1).index,:])
    den2=np.dot(kerasvectors[short_hepmodel.wv.vocab.get(w2).index,:],kerasvectors[short_hepmodel.wv.vocab.get(w2).index,:])
    return num/np.sqrt(den1*den2)
def newdistance(w1,w2): 
    num=np.dot(new_weights[short_hepmodel.wv.vocab.get(w1).index,:],new_weights[short_hepmodel.wv.vocab.get(w2).index,:])
    den1=np.dot(new_weights[short_hepmodel.wv.vocab.get(w1).index,:],new_weights[short_hepmodel.wv.vocab.get(w1).index,:])
    den2=np.dot(new_weights[short_hepmodel.wv.vocab.get(w2).index,:],new_weights[short_hepmodel.wv.vocab.get(w2).index,:])
    return num/np.sqrt(den1*den2)
def olddistance(w1,w2): 
    num=np.dot(short_hepmodel.wv[w1].T,short_hepmodel.wv[w2])
    den1=np.dot(short_hepmodel.wv[w1].T,short_hepmodel.wv[w1])
    den2=np.dot(short_hepmodel.wv[w2].T,short_hepmodel.wv[w2])
    return num/np.sqrt(den1*den2)

In [77]:
short_hepmodel.wv.most_similar(positive=["witten"],topn=15)

[('vafa', 0.8461833000183105),
 ('seiberg', 0.807892918586731),
 ('gromov', 0.7971237897872925),
 ('dijkgraaf', 0.7869737148284912),
 ('gopakumar', 0.7534626722335815),
 ('nekrasov', 0.7298333644866943),
 ('ooguri', 0.7281148433685303),
 ('donaldson', 0.7122985124588013),
 ('dougla', 0.7082318067550659),
 ('refin', 0.7049825191497803),
 ('thoma', 0.6772043704986572),
 ('lmo', 0.6763520240783691),
 ('quiver', 0.668645977973938),
 ('gaiotto', 0.6548724174499512),
 ('equivari', 0.649773120880127)]

In [78]:
def distance(w1,w2):
    print(f"Random distance : {randomdistance(w1,w2)} ")    
    print(f"Word2Vec distance : {olddistance(w1,w2)} ")
    print(f"Our LSTM distance : {newdistance(w1,w2)} ")
def pickledistance(w1,w2,new_weights):
    num=np.dot(new_weights[short_hepmodel.wv.vocab.get(w1).index,:],new_weights[short_hepmodel.wv.vocab.get(w2).index,:])
    den1=np.dot(new_weights[short_hepmodel.wv.vocab.get(w1).index,:],new_weights[short_hepmodel.wv.vocab.get(w1).index,:])
    den2=np.dot(new_weights[short_hepmodel.wv.vocab.get(w2).index,:],new_weights[short_hepmodel.wv.vocab.get(w2).index,:])
    newdist= num/np.sqrt(den1*den2)
    num=np.dot(short_hepmodel.wv[w1].T,short_hepmodel.wv[w2])
    den1=np.dot(short_hepmodel.wv[w1].T,short_hepmodel.wv[w1])
    den2=np.dot(short_hepmodel.wv[w2].T,short_hepmodel.wv[w2])
    olddist= num/np.sqrt(den1*den2)
    num=np.dot(kerasvectors[short_hepmodel.wv.vocab.get(w1).index,:],kerasvectors[short_hepmodel.wv.vocab.get(w2).index,:])
    den1=np.dot(kerasvectors[short_hepmodel.wv.vocab.get(w1).index,:],kerasvectors[short_hepmodel.wv.vocab.get(w1).index,:])
    den2=np.dot(kerasvectors[short_hepmodel.wv.vocab.get(w2).index,:],kerasvectors[short_hepmodel.wv.vocab.get(w2).index,:])
    rand= num/np.sqrt(den1*den2)
    print(f"Random distance : {rand} ")    
    print(f"Word2Vec distance : {olddist} ")
    print(f"Our LSTM distance : {newdist} ")

In [72]:
distance("witten","seiberg")

Random distance : 0.8078928351549151 
Word2Vec distance : 0.8078927397727966 
Our LSTM distance : 0.5743873715400696 


In [73]:
distance("gener","relat")

Random distance : 0.4012932769746991 
Word2Vec distance : 0.4012932777404785 
Our LSTM distance : 0.24563710391521454 


In [74]:
distance("witten","brane")

Random distance : 0.11760112770347647 
Word2Vec distance : 0.11760114133358002 
Our LSTM distance : 0.14298424124717712 


# Load the word vectors to gensim #

In [79]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
glove_file = datapath('/home/ubuntu/project/LSTM/Mar4bigtrainingWord/Weights_v3.txt')
tmp_file = get_tmpfile("/home/ubuntu/project/LSTM/Mar4bigtrainingWord/Weights_v3_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)

In [80]:
our_model = KeyedVectors.load_word2vec_format(tmp_file,)

In [82]:
our_model.similar_by_word("conform",topn=15)

[('threemanifold', 0.5747983455657959),
 ('einsenstein', 0.5551432371139526),
 ('polem', 0.554176926612854),
 ('central', 0.549871563911438),
 ('superconform', 0.5214196443557739),
 ('truism', 0.5194559693336487),
 ('troessaert', 0.5095899105072021),
 ('teichner', 0.5086613893508911),
 ('notivarg', 0.5060545802116394),
 ('asep', 0.503456711769104),
 ('talli', 0.49898064136505127),
 ('reshap', 0.4967975914478302),
 ('poincar', 0.49543410539627075),
 ('contradistinct', 0.49385619163513184),
 ('pankiewicz', 0.49157631397247314)]

# Making abstract vectors #

In [36]:
sent=np.zeros((134296,155,52))
for i in range(134296):
    sent[i]=our_model.wv.vectors[train_X_indices[i]]
    if i%1000==0:
        print(i)

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000


In [41]:
iden=hep_abstracts_limit['id'].values
iden[10343]

'0905.1699'

In [45]:
senter=np.average(sent,axis=1)

# Writing the abstract vectors to the file with a keyword that is given by the arxiv ID #

In [46]:
f = open("sentences_fromwords_v3.txt", "w")
for i in range(134296):
    f.write(f"{iden[i]} ")
    x=senter[i]
    f.write((' '.join(['%0.10f']*x.size)+'\n') % tuple(x))
f.close()

In [38]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

In [48]:
#pickle.dump(our_model,open("modelv3","wb"))

# Now test them #

In [17]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
g = open("/home/ubuntu/project/LSTM/Mar4bigtrainingWord/sentences_fromwords_v3_gensim.txt", "w")
glove_file = datapath('/home/ubuntu/project/LSTM/Mar4bigtrainingWord/sentences_fromwords_v3.txt')
tmp_file = get_tmpfile("/home/ubuntu/project/LSTM/Mar4bigtrainingWord/sentences_fromwords_v3_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
average_sentence_model = KeyedVectors.load_word2vec_format(tmp_file,)
g.close()

# The below code makes paper recommendations using gensim's most similar function#

## It also measure the TF-IDF cosine distance and rejects the recommendation if the distance is less than 0.1 #

In [18]:
tij=hep_abstracts_limit
tij = tij.reset_index(drop=True)
def fastsimplerecommend(string,string1):
    pointer3=hep_abstracts.loc[(hep_abstracts['id'].str.match(string))]
    queryvalue3=pointer3['abstract'].values
    processed_query3=preprocess_documents(queryvalue3)
    test_bow_corpus3 = [hep_dictionary_lim.doc2bow(text) for text in processed_query3]
    vec_test_bow3=hep_tfidf[test_bow_corpus3]
    sims = index[vec_test_bow3[0]]
    ind=tij[tij['id'] == string1].index[0]
    #print(ind)
    if sims[ind]>0.1:
        return (1,sims[ind])
    return (0,0) 

In [19]:
average_sentence_model.similar_by_word("1908.00015",topn=15)

[('1611.07240', 0.9957449436187744),
 ('1007.3865', 0.9955410957336426),
 ('0808.2605', 0.995360255241394),
 ('1311.1160', 0.9953429698944092),
 ('1108.1180', 0.9952816367149353),
 ('hep-th/0003247', 0.9950817823410034),
 ('1510.07770', 0.9950804710388184),
 ('1904.02038', 0.9950685501098633),
 ('hep-ph/0011133', 0.9949870109558105),
 ('hep-ph/9812529', 0.9949166774749756),
 ('1005.3016', 0.9949142336845398),
 ('1911.00496', 0.994881808757782),
 ('hep-th/9803094', 0.9948656558990479),
 ('hep-ph/0007032', 0.9948521852493286),
 ('hep-ph/0505038', 0.9948427081108093)]

In [20]:
def fastlstmrecommender(orig):
    i=1
    j=0
    starter=average_sentence_model.similar_by_word(orig,topn=1000)
    while i<31:
        x=starter[j][0]
        #print(orig,string)
        if fastsimplerecommend(orig,x)[0]==1:
            print(f"\n {i}) With TFIDF score {fastsimplerecommend(orig,x)[1]} and GPT-2 rank {j} ")
            print(f" Paper title : {hep_abstracts.loc[hep_abstracts['id'] == x].title.values}. \n Abstract : {hep_abstracts.loc[hep_abstracts['id'] == x].abstract.values}. \n Authors : {datapd_hep_th.loc[hep_abstracts['id'] == x].authors.values}. \n Arxiv ID : {hep_abstracts.loc[hep_abstracts['id'] == x].id.values} \n \n ")
            i=i+1
        j=j+1
def lambdalstmrecommender(orig):
    i=1
    j=0
    starter=average_sentence_model.similar_by_word(orig,topn=1000)
    while i<31:
        x=starter[j][0]
        #print(orig,string)
        if fastsimplerecommend(orig,x)[0]==1:
            print(f"\"{hep_abstracts.loc[hep_abstracts['id'] == x].id.values[0]}\",",) 
            i=i+1
        j=j+1

In [21]:
lambdalstmrecommender("1212.3616") #Jared et al 

"hep-th/0105283",
"1802.04390",
"0912.2726",
"1910.07424",
"2005.09105",
"1809.00551",
"1712.06622",
"1211.5838",
"1702.08471",
"hep-th/0601148",
"2003.07361",
"hep-th/0201258",
"1203.5129",
"1911.11487",
"1808.08155",
"1207.3112",
"1811.00467",
"1812.06479",
"1310.5078",
"1709.00008",
"1306.1930",
"1504.00772",
"1408.3629",
"1512.05362",
"1411.7941",
"0905.2211",
"hep-th/9310130",
"1404.5864",
"1803.05086",
"hep-th/0409058",


In [127]:
lambdalstmrecommender("1304.4926") #Lewkowycz Maldacena

"1805.12145",
"1303.1884",
"1803.06164",
"1403.3416",
"1408.6932",
"1806.02209",
"1705.08319",
"1807.04294",
"1404.3105",
"1310.1839",
"1705.01561",
"1503.03542",
"hep-th/9809035",
"1412.8465",
"1912.04837",
"hep-th/0203219",
"1410.5763",
"1109.0544",
"hep-th/0002145",
"0804.0632",
"1709.03205",
"hep-th/9901148",
"1504.00106",
"1902.02504",
"1201.1702",
"1411.1887",
"hep-th/0201036",
"0711.1221",
"2009.11595",
"1010.3700",


In [128]:
lambdalstmrecommender("1908.10996") #Maldacena et al

"hep-th/0610140",
"1608.08304",
"1911.09666",
"2004.13857",
"1205.5036",
"0806.4373",
"1412.0802",
"1512.06855",
"1305.3448",
"0704.0507",
"gr-qc/0505068",
"hep-th/0605068",
"1402.5127",
"hep-th/9607235",
"1305.3757",
"1502.00583",
"0705.3150",
"1905.04317",
"hep-th/0606069",
"1612.04857",
"hep-th/0011286",
"0911.5070",
"1603.07976",
"1912.02210",
"quant-ph/0311049",
"0804.2123",
"gr-qc/9909061",
"2010.07756",
"1406.1804",
"hep-th/0302170",


In [129]:
lambdalstmrecommender("1805.00098") #DSD

"2010.04726",
"1803.04938",
"1005.4516",
"1609.02165",
"1610.05308",
"hep-th/9403026",
"1611.00763",
"2005.06343",
"1909.05775",
"hep-th/9604044",
"1403.1622",
"1012.3740",
"1711.04371",
"1807.05941",
"1804.09334",
"1801.04208",
"1611.05577",
"1912.04105",
"1308.1726",
"1810.07199",
"2011.09250",
"1206.3129",
"1512.02532",
"hep-th/9411100",
"1504.01737",
"1812.02226",
"2002.03865",
"hep-th/0109064",
"1212.4103",
"1601.02883",


In [130]:
lambdalstmrecommender("1606.01857") #Yang et al 

"1611.09381",
"2009.03948",
"1005.1973",
"hep-th/0608182",
"1712.09826",
"1511.06842",
"1307.7732",
"hep-th/0506177",
"1410.4466",
"1012.1040",
"1106.2291",
"1808.08239",
"1310.4549",
"1704.06076",
"1411.6405",
"1408.2538",
"1912.12996",
"1511.08816",
"1809.04081",
"1606.04123",
"1303.7211",
"hep-th/9805146",
"1602.07177",
"1105.4862",
"hep-th/0112058",
"hep-th/0607111",
"0812.2234",
"hep-th/0703046",
"1203.2636",
"1108.1205",


In [131]:
lambdalstmrecommender("0712.0155") # Maloney Witten

"1307.7732",
"hep-th/9504033",
"0711.0133",
"0910.5596",
"1504.05044",
"0706.0610",
"hep-th/0312038",
"1411.6505",
"1604.00383",
"1703.10178",
"hep-th/9412209",
"2011.06026",
"hep-th/0210017",
"hep-th/0203219",
"1908.02470",
"2009.03948",
"1912.00029",
"hep-th/9803180",
"1803.10775",
"hep-th/9204072",
"1404.0033",
"hep-th/0412223",
"1507.04757",
"1403.2721",
"hep-th/0610163",
"0802.2257",
"1209.5396",
"hep-th/0506176",
"hep-th/0301239",
"hep-th/0003144",


In [132]:
lambdalstmrecommender("1703.00278") # Inversion formula

"1510.08772",
"1905.01355",
"1904.00032",
"2006.08221",
"1510.02464",
"0901.4864",
"1706.02388",
"1802.06889",
"0704.1417",
"1407.6429",
"1806.08181",
"1910.04661",
"1611.05797",
"1305.4604",
"1002.4613",
"1402.6127",
"1104.4783",
"1210.7705",
"1706.08456",
"1605.06039",
"2007.11647",
"2002.12091",
"hep-lat/9709079",
"1310.4778",
"hep-th/9903196",
"hep-ph/0306250",
"1302.3539",
"1605.08087",
"1603.05150",
"1607.03123",


In [29]:
lambdalstmrecommender("1405.5137") # HKS

"1412.1276",
"1505.04804",
"1511.05179",
"1406.0520",
"0704.0140",
"1608.00328",
"1407.6429",
"1307.6607",
"1107.2370",
"1711.09913",
"1310.3676",
"hep-th/0105117",
"2010.00407",
"1605.02835",
"1811.11189",
"hep-th/0101083",
"1606.04537",
"hep-th/9907129",
"cond-mat/0008216",
"1306.4974",
"1506.03772",
"1303.0741",
"1405.0015",
"0907.5542",
"1412.5205",
"hep-th/0002250",
"1108.0677",
"1604.07830",
"1303.6955",
"1608.01283",


In [135]:
lambdalstmrecommender("1611.03470") # Verlinde

"1701.06340",
"1102.1734",
"1808.07760",
"1910.06675",
"hep-th/0112150",
"1909.11402",
"1801.02714",
"1109.0544",
"1612.00141",
"1806.10807",
"0812.0152",
"1106.1533",
"1809.04081",
"2004.01192",
"1602.06786",
"hep-th/0602215",
"1401.4998",
"1811.07965",
"2011.04664",
"1609.08912",
"1806.00306",
"1804.07544",
"1609.06300",
"1909.02357",
"1211.4506",
"hep-th/0311004",
"1707.09663",
"hep-th/0505046",
"hep-th/0101118",
"1210.0284",


In [22]:
lambdalstmrecommender("hep-th/0310285")  #Confinement

"0812.3903",
"cond-mat/0311222",
"0705.3388",
"1206.6375",
"0708.3199",
"1501.07615",
"2008.13446",
"hep-th/0311177",
"1808.07056",
"0906.4959",
"1102.1900",
"hep-ph/0206181",
"2008.02579",
"1108.0683",
"1603.00553",
"1003.1302",
"1707.05307",
"1302.4898",
"1409.8305",
"1501.04033",
"1709.01801",
"1002.3823",
"1004.3570",
"1910.12856",
"1406.5498",
"hep-th/0505038",
"hep-th/9512048",
"1410.3390",
"0806.0558",
"1706.02702",


In [23]:
lambdalstmrecommender("1411.7041")  #Dong 1411

"1906.05501",
"1811.05171",
"2008.01755",
"1408.3705",
"1906.03669",
"1912.02799",
"1909.11402",
"1606.04537",
"2004.15010",
"2003.11870",
"1912.05649",
"0907.0151",
"1806.10560",
"1902.10161",
"2010.01907",
"1504.06632",
"1407.2900",
"1612.04373",
"1507.00779",
"2005.05962",
"1809.01355",
"1702.01748",
"1607.03605",
"1612.00017",
"1810.02055",
"1703.07780",
"1903.11244",
"1209.4641",
"2005.05971",
"0904.3544",


In [24]:
lambdalstmrecommender("1601.05416")  #Dong 1601

"1310.6334",
"1812.05268",
"1808.05963",
"1604.01772",
"1906.04745",
"1704.00187",
"hep-th/0209189",
"1905.06920",
"1408.0415",
"1512.06431",
"1803.10539",
"1306.4324",
"1605.05751",
"1403.1393",
"1811.03113",
"1311.4516",
"1703.05445",
"0909.5617",
"1703.01759",
"2011.00407",
"0801.2863",
"1805.05398",
"1511.07194",
"1701.02319",
"1703.09222",
"1908.02044",
"1606.07628",
"2007.05365",
"1601.07616",
"1802.01040",


In [25]:
lambdalstmrecommender("1605.08072")  #Faulkner

"1802.10103",
"1803.01874",
"1705.01486",
"2008.07532",
"1604.05308",
"1705.08453",
"1607.07506",
"1611.03470",
"2010.15813",
"1703.00456",
"2004.07870",
"1809.00026",
"1511.06713",
"1909.11402",
"2004.07242",
"1611.01846",
"1412.0687",
"1806.10807",
"1811.05382",
"1407.8171",
"1802.04278",
"0907.1625",
"2006.01835",
"2006.10740",
"1512.08233",
"math-ph/9805013",
"1908.09939",
"1707.03825",
"2005.05962",
"1406.2991",


In [26]:
lambdalstmrecommender("1405.5137")  #HKS

"1412.1276",
"1505.04804",
"1511.05179",
"1406.0520",
"0704.0140",
"1608.00328",
"1407.6429",
"1307.6607",
"1107.2370",
"1711.09913",
"1310.3676",
"hep-th/0105117",
"2010.00407",
"1605.02835",
"1811.11189",
"hep-th/0101083",
"1606.04537",
"hep-th/9907129",
"cond-mat/0008216",
"1306.4974",
"1506.03772",
"1303.0741",
"1405.0015",
"0907.5542",
"1412.5205",
"hep-th/0002250",
"1108.0677",
"1604.07830",
"1303.6955",
"1608.01283",


In [27]:
lambdalstmrecommender("0712.2456")  #fluid

"1302.5060",
"1206.6106",
"1910.04484",
"1012.1040",
"1406.2328",
"1306.4501",
"1609.08912",
"1310.0819",
"1704.07410",
"1405.3965",
"1912.05746",
"1106.1533",
"1805.00284",
"1806.00306",
"0809.2596",
"1704.05141",
"0809.4928",
"1704.06076",
"2008.13699",
"0812.1818",
"1312.4953",
"hep-th/0306031",
"hep-th/0303174",
"gr-qc/0612191",
"1311.4794",
"1403.7639",
"1304.6305",
"1912.09402",
"1710.04221",
"hep-th/0610113",


In [28]:
lambdalstmrecommender("1604.07818")  #SYK

"1905.08256",
"1205.6157",
"2007.12188",
"1610.01569",
"1902.07407",
"1707.01740",
"1706.02231",
"hep-th/9711167",
"1508.06668",
"1708.06446",
"hep-th/9305031",
"cond-mat/0701501",
"hep-ph/0510012",
"1905.00026",
"1802.00364",
"gr-qc/0212003",
"1410.0298",
"1903.00478",
"1701.07426",
"1711.11042",
"1905.07694",
"hep-th/9402109",
"2006.07317",
"1705.03956",
"hep-th/9612053",
"1808.07455",
"1407.4497",
"2008.03308",
"hep-lat/9707027",
"hep-th/9703107",


In [134]:
lambdalstmrecommender("1611.04650") # Saad Cotler Phil

"1907.04289",
"1706.07439",
"1611.04592",
"1809.07228",
"1805.01051",
"1605.08124",
"0712.3574",
"1809.01671",
"1711.05556",
"1812.04770",
"1211.3425",
"2009.00069",
"1506.04795",
"1806.06840",
"1708.08822",
"hep-th/0201165",
"1702.04350",
"1802.06796",
"1209.2417",
"1410.3472",
"1701.07783",
"1206.2630",
"0808.0530",
"1705.07597",
"hep-th/0508044",
"hep-ph/0006333",
"hep-th/0003208",
"1304.0007",
"1110.0803",
"0808.1725",


# Human checks #

In [116]:
fastlstmrecommender("1212.3616") #Jared


 1) With TFIDF score 0.32867228984832764 and GPT-2 rank 0 
 Paper title : ['Operator product expansion in SL(2) conformal field theory']. 
 Abstract : ['  In the conformal field theories having affine SL(2) symmetry, we study the\noperator product expansion (OPE) involving primary fields in highest weight\nrepresentations. For this purpose, we analyze properties of primary fields with\ndefinite SL(2) weights, and calculate their two- and three-point functions.\nUsing these correlators, we show that the correct OPE is obtained when one of\nthe primary fields belongs to the degenerate highest weight representation. We\nbriefly comment on the OPE in the SL(2,R) WZNW model.\n']. 
 Authors : ['Kazuo Hosomichi and Yuji Satoh']. 
 Arxiv ID : ['hep-th/0105283'] 
 
 

 2) With TFIDF score 0.45168179273605347 and GPT-2 rank 1 
 Paper title : ['Veneziano Amplitude of Vasiliev Theory']. 
 Abstract : ["  We compute the four-point function of scalar operators in CFTs with weakly\nbroken higher spin


 12) With TFIDF score 0.33895793557167053 and GPT-2 rank 24 
 Paper title : ['Bootstrapping the 3d Ising twist defect']. 
 Abstract : ['  Recent numerical results point to the existence of a conformally invariant\ntwist defect in the critical 3d Ising model. In this note we show that this\nfact is supported by both epsilon expansion and conformal bootstrap\ncalculations. We find that our results are in good agreement with the numerical\ndata. We also make new predictions for operator dimensions and OPE coefficients\nfrom the bootstrap approach. In the process we derive universal bounds on\none-dimensional conformal field theories and conformal line defects.\n']. 
 Authors : ['Davide Gaiotto, Dalimil Mazac, Miguel F. Paulos']. 
 Arxiv ID : ['1310.5078'] 
 
 

 13) With TFIDF score 0.20933914184570312 and GPT-2 rank 25 
 Paper title : ['A tauberian theorem for the conformal bootstrap']. 
 Abstract : ['  For expansions in one-dimensional conformal blocks, we provide a rigorous\nlink betw


 22) With TFIDF score 0.36691975593566895 and GPT-2 rank 59 
 Paper title : ['The Lorentzian inversion formula and the spectrum of the 3d O(2) CFT']. 
 Abstract : ['  We study the spectrum and OPE coefficients of the three-dimensional critical\nO(2) model, using four-point functions of the leading scalars with charges 0,\n1, and 2 ($s$, $\\phi$, and $t$). We obtain numerical predictions for low-twist\nOPE data in several charge sectors using the extremal functional method. We\ncompare the results to analytical estimates using the Lorentzian inversion\nformula and a small amount of numerical input. We find agreement between the\nanalytic and numerical predictions. We also give evidence that certain scalar\noperators lie on double-twist Regge trajectories and obtain estimates for the\nleading Regge intercepts of the O(2) model.\n']. 
 Authors : ['Junyu Liu, David Meltzer, David Poland, David Simmons-Duffin']. 
 Arxiv ID : ['2007.07914'] 
 
 

 23) With TFIDF score 0.22024957835674286 an

In [ ]:
fastlstmrecommender("1606.01857")

In [115]:
fastlstmrecommender("1908.10996") #Raghu Juan


 1) With TFIDF score 0.23111343383789062 and GPT-2 rank 0 
 Paper title : ['Can Hawking temperatures be negative ?']. 
 Abstract : ["  It has been widely believed that the Hawking temperature for a black hole is\n$uniquely$ determined by its metric and $positive$. But, I find that this does\n``not'' seem to be true in the recently discovered black holes which include\nthe exotic black holes and the black holes in the three-dimensional higher\ncurvature gravities. I show that the Hawking temperatures, which are measured\nby the quantum fields in thermal equilibrium with the black holes, are $not$\nthe usual Hawking temperature but the $new$ temperatures that have been\nproposed recently and can be $negative$. The associated new entropy formulae,\nwhich are defined by the first law of thermodynamics, versus the black hole\nmasses show some genuine effects of the black holes which do not occur in the\nspin systems. Some cosmological implications and physical origin of the\ndiscrepancy wi


 9) With TFIDF score 0.4030921459197998 and GPT-2 rank 11 
 Paper title : ['From qubits to black holes: entropy, entanglement and all that']. 
 Abstract : ['  Entropy plays a crucial role in characterization of information and\nentanglement, but it is not a scalar quantity and for many systems it is\ndifferent for different relativistic observers. Loop quantum gravity predicts\nthe Bekenstein-Hawking term for black hole entropy and logarithmic correction\nto it. The latter originates in the entanglement between the pieces of spin\nnetworks that describe black hole horizon. Entanglement between gravity and\nmatter may restore the unitarity in the black hole evaporation process. If the\ncollapsing matter is assumed to be initially in a pure state, then entropy of\nthe Hawking radiation is exactly the created entanglement between matter and\ngravity.\n']. 
 Authors : ['Daniel R. Terno']. 
 Arxiv ID : ['gr-qc/0505068'] 
 
 

 10) With TFIDF score 0.2915576696395874 and GPT-2 rank 18 
 Pap


 18) With TFIDF score 0.2801590859889984 and GPT-2 rank 31 
 Paper title : ['A new spin on entanglement entropy']. 
 Abstract : ['  We argue that the usual notions of thermodynamic and entanglement entropy\nhave novel analogs in the context of higher spin theories. In particular, the\nWald and Ryu-Takayanagi formulas have natural higher spin extensions that we\nwork out and study. On the CFT side, just as standard entanglement entropy in\nCFT_2 can be computed from twist field correlators, we demonstrate that by\nintroducing corresponding operators carrying higher spin charge we can\nprecisely reproduce our results from the bulk.\n  We also show that the first law for entanglement entropy implies the\nlinearized field equations for the metric and higher spin fields, generalizing\nrecent work on deriving the linearized Einstein equations from the first law.\n']. 
 Authors : ['Eliot Hijano and Per Kraus']. 
 Arxiv ID : ['1406.1804'] 
 
 

 19) With TFIDF score 0.2314385026693344 and GPT


 27) With TFIDF score 0.27239707112312317 and GPT-2 rank 53 
 Paper title : ['Islands outside the horizon']. 
 Abstract : ['  We consider an AdS$_2$ black hole in equilibrium with a bath, which we take\nto have a dual description as (0+1)-dimensional quantum mechanical system\ncoupled to a (1+1)-dimensional field theory serving as the bath. We compute the\nentropies of both the quantum mechanical degrees of freedom and of the bath\nseparately, while allowing contributions from entanglement wedge "islands". We\nfind situations where the island extends {\\it outside} the black hole horizon.\nThis suggests possible causality paradoxes which we show are avoided because of\nthe quantum focusing conjecture. Finally, we formulate a version of the\ninformation paradox for a black hole in contact with a bath in the\nHartle-Hawking state, and demonstrate the role of islands in resolving this\nparadox.\n']. 
 Authors : ['Ahmed Almheiri, Raghu Mahajan, Juan Maldacena']. 
 Arxiv ID : ['1910.11077'

# Finally we end by mentioning the code for using the hidden states of generating abstract vectors. As mentioned before, this method does not lead to meaningful embeddings #

In [ ]:
X_shuffled.shape
exp_size=1024
def BatchGeneratortest(train_X_indices):
    for i in range(1,132):
        X_train = train_X_indices[exp_size*(i-1):exp_size*i,:]
        yield (X_train)
    X_train = train_X_indices[exp_size*i:-1,:]
    yield (X_train)
sentence=[]
context=[]

In [ ]:
i=0
for (X_train) in BatchGeneratortest(train_X_indices):
    m = X_train.shape[0]
    s0 = np.zeros((m, n_s))
    c0 = np.zeros((m, n_s))
    i=i+1
    print(f" \n Metabatch number --------------------- {i} \n ")
    x=kerasmodel[1].predict([X_train, s0, c0])[1]
    y=kerasmodel[1].predict([X_train, s0, c0])[2]
    sentence.append(x)
    context.append(y)

In [ ]:
sentence1=np.asarray(sentence[0:130])
context1=np.asarray(context[0:130])

In [ ]:
reshaped_sentence_raw=np.asarray(sentence1).reshape(133120,12,64)
reshaped_context_raw=context1.reshape(133120,12,128)

In [ ]:
average_sentence_raw=np.average(reshaped_sentence_raw,axis=1)
average_context_raw=np.average(reshaped_context_raw,axis=1)
first_sentence_raw=reshaped_sentence_raw[:,0,:]
first_context_raw=reshaped_context_raw[:,0,:]

In [ ]:
iden=hep_abstracts_limit['id'].values
iden[10343]

In [ ]:
f = open("ModelsMar3/sentences_average_v2.txt", "w")
for i in range(133120):
    f.write(f"{iden[i]} ")
    x=average_sentence_raw[i,:]
    f.write((' '.join(['%0.10f']*x.size)+'\n') % tuple(x))
f.close()

In [ ]:
f = open("ModelsMar3/sentences_first_v2.txt", "w")
for i in range(133120):
    f.write(f"{iden[i]} ")
    x=first_sentence_raw[i,:]
    f.write((' '.join(['%0.10f']*x.size)+'\n') % tuple(x))
f.close()

In [ ]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

The below model uses the average of all the hidden states S.

In [ ]:
g = open("ModelsMar3/sentences_average_v2_gensim.txt", "w")
glove_file = datapath('/home/ubuntu/ModelsMar3/sentences_average_v2.txt')
tmp_file = get_tmpfile("/home/ubuntu/ModelsMar3/sentences_average_v2_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
average_sentence_model = KeyedVectors.load_word2vec_format(tmp_file,)
g.close()

In [ ]:
average_sentence_model.wv.similar_by_word("1908.10996",topn=25)

The below model uses the first hidden state S<1>.

In [ ]:
g = open("ModelsMar3/sentences_first_v2_gensim.txt", "w")
glove_file = datapath('/home/ubuntu/ModelsMar3/sentences_first_v2.txt')
tmp_file = get_tmpfile("/home/ubuntu/ModelsMar3/sentences_first_v2_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
first_sentence_model = KeyedVectors.load_word2vec_format(tmp_file,)
g.close()

In [ ]:
first_sentence_model.wv.similar_by_word("1908.10996",topn=25)

# No new code below. Only testing #

In [88]:
distance("holographi","maldacena")

Random distance : 0.16314263373452945 
Word2Vec distance : 0.43044283986091614 
Our LSTM distance : 0.7871480584144592 


In [89]:
distance("witten","string")

Random distance : -0.3352357209886495 
Word2Vec distance : 0.2427290976047516 
Our LSTM distance : 0.8788047432899475 


In [90]:
distance("diagram","feynman")

Random distance : 0.14352409503724678 
Word2Vec distance : 0.5671595335006714 
Our LSTM distance : 0.8831663727760315 


In [91]:
distance("brane","witten")

Random distance : -0.23661674825714057 
Word2Vec distance : 0.11760114133358002 
Our LSTM distance : 0.9001157283782959 


In [92]:
distance("wilson","rg")

Random distance : 0.2071838269895777 
Word2Vec distance : 0.06880977749824524 
Our LSTM distance : 0.14451353251934052 


In [93]:
distance("renormaliz","quantum")

Random distance : -0.10235812366783505 
Word2Vec distance : 0.17221076786518097 
Our LSTM distance : 0.9171896576881409 


In [94]:
distance("graviti","nonrenormaliz")

Random distance : -0.10621026902414282 
Word2Vec distance : 0.22952578961849213 
Our LSTM distance : -0.13791470229625702 


In [95]:
distance("pauli","heisenberg")

Random distance : 0.21725816887317384 
Word2Vec distance : 0.37719425559043884 
Our LSTM distance : 0.9705891609191895 


In [96]:
distance("isi","fix")

Random distance : -0.07691638769477205 
Word2Vec distance : -0.1820271611213684 
Our LSTM distance : 0.023427601903676987 


In [97]:
distance("einstein","quantum")

Random distance : -0.03996431003842751 
Word2Vec distance : 0.0982431173324585 
Our LSTM distance : 0.9848122000694275 


In [98]:
distance("graviti","quantum")

Random distance : 0.03008633381230795 
Word2Vec distance : -0.02838786505162716 
Our LSTM distance : 0.9894531965255737 


In [99]:
distance("string","graviti")

Random distance : -0.09763714679433107 
Word2Vec distance : -0.050817616283893585 
Our LSTM distance : 0.9843711256980896 


In [100]:
distance("isi","conform")

Random distance : -0.01920914502465594 
Word2Vec distance : -0.12263837456703186 
Our LSTM distance : 0.022416891530156136 


In [101]:
distance("isi","cft")

Random distance : 0.001091574923656666 
Word2Vec distance : -0.20761822164058685 
Our LSTM distance : 0.030352793633937836 


In [102]:
distance("mellin","bootstrap")

Random distance : -0.020424609488757274 
Word2Vec distance : 0.42293426394462585 
Our LSTM distance : 0.11947142332792282 


In [148]:
distance("string","theori")

Random distance : 0.1785067186563328 
Word2Vec distance : 0.13560254871845245 
Our LSTM distance : 0.9921423196792603 


In [149]:
distance("quantum","theori")

Random distance : -0.1708688062381955 
Word2Vec distance : 0.16729344427585602 
Our LSTM distance : 0.9938627481460571 


In [150]:
distance("quark","color")

Random distance : 0.07184288424688501 
Word2Vec distance : 0.6671873927116394 
Our LSTM distance : 0.8904553055763245 


In [151]:
distance("gaug","confin")

Random distance : -0.09706513182803737 
Word2Vec distance : 0.4093013107776642 
Our LSTM distance : 0.9636445045471191 


In [152]:
distance("wilson","confin")

Random distance : -0.09926400211216445 
Word2Vec distance : 0.18497388064861298 
Our LSTM distance : 0.9443263411521912 


In [153]:
distance("quark","confin")

Random distance : 0.18325472491700956 
Word2Vec distance : 0.6506814360618591 
Our LSTM distance : 0.9688527584075928 


In [130]:
our_model.wv.similar_by_word("conform",topn=15)

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('lcr', 0.732690691947937),
 ('blason', 0.7310201525688171),
 ('simi', 0.7129940986633301),
 ('reanalysi', 0.7126487493515015),
 ('fp', 0.7118764519691467),
 ('depress', 0.7115901708602905),
 ('hom', 0.7081080675125122),
 ('lamm', 0.7070679068565369),
 ('reestim', 0.6944543719291687),
 ('supersymetri', 0.6909819841384888),
 ('intens', 0.6893722414970398),
 ('ressum', 0.6885448694229126),
 ('infraparticl', 0.6870040893554688),
 ('anisotropi', 0.6789699792861938),
 ('dslash', 0.6778581142425537)]

In [106]:
pickledistance("witten","brane",weights6)

Random distance : -0.23661674825714057 
Word2Vec distance : 0.11760114133358002 
Our LSTM distance : 0.371532142162323 


In [109]:
pickledistance("witten","brane",weights3)

Random distance : -0.23661674825714057 
Word2Vec distance : 0.11760114133358002 
Our LSTM distance : -0.7255784273147583 


In [110]:
pickledistance("witten","brane",weights1)

Random distance : -0.23661674825714057 
Word2Vec distance : 0.11760114133358002 
Our LSTM distance : -0.5968130230903625 
